
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [2]:
import pandas as pd

df = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin-1')
df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [3]:
df = df[df['Sentiment'] != 'Neutral']
df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
...,...,...,...,...,...,...
41150,44949,89901,OHIO,14-04-2020,I never that weÂd be in a situation &amp; wor...,Positive
41151,44950,89902,NaN,14-04-2020,@MrSilverScott you are definitely my man. I fe...,Extremely Positive
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive


In [4]:
X = df['OriginalTweet'].to_numpy()
X

array(['advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order',
       'Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P',
       "My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",
       ...,
       'Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?',
       'You know itÂ\x92s getting tough when @KameronWilds  is rationing toilet paper #coronavirus #toiletpaper @krog

In [5]:
Y = df['Sentiment']
Y

1                  Positive
2                  Positive
3                  Positive
4        Extremely Negative
5                  Positive
                ...        
41150              Positive
41151    Extremely Positive
41153    Extremely Negative
41154              Positive
41156              Negative
Name: Sentiment, Length: 33444, dtype: object

In [6]:
Y.unique()

array(['Positive', 'Extremely Negative', 'Negative', 'Extremely Positive'],
      dtype=object)

In [7]:
Y = Y.apply(lambda x: 1 if x in ['Positive', 'Extremely Positive'] else 0)
Y = Y.to_numpy()
Y

array([1, 1, 1, ..., 0, 1, 0], shape=(33444,))

In [8]:
train_size = int(len(X) * 0.6)
test_size = int(len(X) * 0.15)

X_train, Y_train = X[:train_size], Y[:train_size]
X_test, Y_test = X[train_size:train_size + test_size], Y[train_size:train_size + test_size]
X_valid, Y_valid = X[train_size + test_size:], Y[train_size + test_size:]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create a pipeline that includes both CountVectorizer and LogisticRegression
model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(verbose=1, max_iter=5000))
    # Adjust max_iter if needed
])

# Train the model
model.fit(X_train, Y_train)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        45371     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  4.18868D-02


 This problem is unconstrained.



At iterate   50    f=  2.01773D-01    |proj g|=  3.54935D-03

At iterate  100    f=  1.92569D-01    |proj g|=  4.47796D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
45371    134    151      1     0     0   6.341D-05   1.921D-01
  F =  0.19210738717715287     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression(max_iter=5000, verbose=1))])

In [10]:
from sklearn.metrics import precision_score, recall_score

# Predict on validation data
Y_valid_pred = model.predict(X_valid)

# Calculate precision and recall
precision_pos = precision_score(Y_valid, Y_valid_pred, pos_label=1)
recall_pos = recall_score(Y_valid, Y_valid_pred, pos_label=1)
precision_neg = precision_score(Y_valid, Y_valid_pred, pos_label=0)
recall_neg = recall_score(Y_valid, Y_valid_pred, pos_label=0)

print("Precision (Positive):", precision_pos)
print("Recall (Positive):", recall_pos)
print("Precision (Negative):", precision_neg)
print("Recall (Negative):", recall_neg)

Precision (Positive): 0.8782414910858996
Recall (Positive): 0.9070935342121783
Precision (Negative): 0.8704028021015762
Recall (Negative): 0.8322634663689645


In [11]:
Y_test_pred = model.predict(X_test)

precision_0 = precision_score(Y_test, Y_test_pred, pos_label=0)
precision_1 = precision_score(Y_test, Y_test_pred, pos_label=1)
recall_0 = recall_score(Y_test, Y_test_pred, pos_label=0)
recall_1 = recall_score(Y_test, Y_test_pred, pos_label=1)

In [12]:
recall_1

0.8797491700479528

In [13]:
# Part 4

def cost(model, threshold, X, Y):
    # Get predicted probabilities for class 1 (positive sentiment)
    Y_pred_proba = model.predict_proba(X)[:, 1]

    total_cost = 0
    n = len(Y)

    for true_label, prob in zip(Y, Y_pred_proba):

        if true_label == 1 and prob < threshold:
            total_cost += 1  # Positive tweet classified as negative

        elif true_label == 0 and prob >= threshold:
            total_cost += 5  # Negative tweet classified as positive

    return total_cost / n



In [16]:
import numpy as np

# Try different thresholds
thresholds = np.linspace(0, 1, 15)
costs = [cost(model, t, X_test, Y_test) for t in thresholds]

optimal_threshold = thresholds[np.argmin(costs)]
cost_at_optimal_threshold = cost(model, optimal_threshold, X_test, Y_test)

In [29]:
# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
n = len(Y_valid)
confidence_level = 0.99
z_score = 2.576  # For 99% confidence
error_bound = np.sqrt(25 * np.log(2 / 0.01) / (2 * n))
# error_bound = z_score * math.sqrt(cost_at_optimal_threshold_valid * (1 - cost_at_optimal_threshold_valid) / n)

lower_bound = cost_at_optimal_threshold_valid - error_bound
upper_bound = cost_at_optimal_threshold_valid + error_bound

cost_interval_valid = (lower_bound, upper_bound)



In [55]:
optimal_threshold

np.float64(0.7857142857142857)

In [56]:
print(cost_at_optimal_threshold_valid, cost_interval_valid)

0.27876106194690264 (np.float64(0.2609619266229047), np.float64(0.2965601972709006))


In [60]:
import numpy as np


# Part 7
def compute_variance_and_interval(model, threshold, X_val, Y_val, confidence=0.99):
    # Step 1: Compute C for the validation set
    Y_pred_proba = model.predict_proba(X_val)[:, 1]
    C = [
        (1 - (pred >= threshold)) * true_label + 5 * (1 - true_label) * (pred >= threshold)
        for true_label, pred in zip(Y_val, Y_pred_proba)
    ]

    # Step 2: Compute empirical variance
    C = np.array(C)
    mean_C = np.mean(C)
    print(mean_C)
    variance_of_C = np.var(C, ddof=1)  # ddof=1 for unbiased variance

    # Step 3: Compute confidence interval using Bennett's inequality
    n = len(C)
    epsilon = np.sqrt((2 * variance_of_C * np.log(1 / (1 - confidence))) / n)  # Hoeffding-like bound simplification

    # Confidence interval
    interval_of_C = (mean_C - epsilon, mean_C + epsilon)

    return variance_of_C, interval_of_C


variance_of_C, interval_of_C = compute_variance_and_interval(model, optimal_threshold, X_valid, Y_valid)


0.27876106194690264


In [61]:
interval_of_C

(np.float64(0.25078471379084244), np.float64(0.30673741010296285))